<a href="https://colab.research.google.com/github/qsquentinsmith/minnesota_real_estate_analysis/blob/main/real_estate_metadata_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Meta Data from Real Estate Website

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import matplotlib.pyplot as plt
%matplotlib inline

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Call data back to read csv
# Resources: https://github.com/JacobSampson/mls_scraper. Forked repo to https://github.com/qsquentinsmith/mls_scraper to add more features
df_properties = pd.read_csv('https://raw.githubusercontent.com/qsquentinsmith/mls_scraper/main/output/properties.csv')

In [ ]:
# copy data frame for description analysis
df_properties_description = df_properties.copy()
df_properties_description = df_properties_description[['pid', 'description']]

In [ ]:
# no truncation
pd.set_option('display.max_colwidth', None)

In [ ]:
# Display shape
df_properties.shape

(138868, 48)

## Remove Unnecessary Data



We will first start by removing any unecessary data. This can include uninformative, repetitive or irrelevant data. 

When looking at the column list we see there is some repetitive data in location data. Given the context of real estate we will keep the address and get rid of the latitude and longitude data. 

Lastly, the goal of this project is to analyze housing prices of the twin cities area. So we will examine the property_type column and keep only family/single housing. We will then only keep the zip codes that belong within the twin cities metro.

In [ ]:
# remove latitude, longitude, description, image columns because it is redundant or not needed
df_properties = df_properties.drop(['lat', 'lng', 'description', ' images'], axis=1)
# We can also remove the half bath, full bath, and 3/4 bath and just keep num_bath
df_properties = df_properties.drop(['half_baths', 'quarter_baths', 'threequarter_baths', 'full_baths'], axis=1)
# We can also remove the mls
df_properties = df_properties.drop(['mls'], axis=1)

In [ ]:
# Let us check the unique values of property_type
print(df_properties.property_type.unique())

[nan 'SingleFamily' 'Condo/Townhome' 'Multi-family' 'Lot/Land' 'Farm'
 'Commercial']


In [ ]:
# keep property types for family homes, fingle family, multi-family, condo/townhome, 
df_properties = df_properties[df_properties.property_type != 'Farm']
df_properties = df_properties[df_properties.property_type != 'Lot/Land']
df_properties = df_properties[df_properties.property_type != 'Commercial']

# Let recheck unique values
print(df_properties.property_type.unique())
print(df_properties.shape)
#Note we will be dealing with nan later

[nan 'SingleFamily' 'Condo/Townhome' 'Multi-family']
(129266, 39)


From wikipedia the twin cities metropolitan area includes 15 counties. 13 of those 15 counties belong to Minnesota. We will be keeping the data for these 13 counties listed.

Twin Cities Counties: Hennepin, Ramsey, Dakota, Anoka, Washington, Scott, Wright, Carver, Sherburne, Chisago, Isanti, Le Sueur, Mille Lacs

In [ ]:
print(df_properties.county.unique())

[nan 'Hennepin' 'Washington' 'Dakota' 'Anoka' 'Scott' 'Ramsey' 'Wright'
 'Chisago' 'Sherburne' 'McLeod' 'Carver' 'Pine' 'Isanti' 'Douglas'
 'Benton' 'BlueEarth' 'Nicollet' 'St.Croix' 'Faribault' 'Rice' 'Renville'
 'Hubbard' 'CrowWing' 'St.Louis' 'Pierce' 'Cass' 'Bayfield' 'Olmsted'
 'Sawyer' 'Polk' 'Itasca' 'Stearns' 'Lake' 'Washburn' 'OtterTail' 'Becker'
 'Buffalo' 'Kandiyohi' 'CerroGordo' 'Chippewa' 'Mahnomen' 'LeSueur'
 'Goodhue' 'LaCrosse' 'Morrison' 'Koochiching' 'Barron' 'Wabasha'
 'Ashland' 'Sibley' 'Trempealeau' 'EauClaire' 'Meeker' 'Aitkin' 'Todd'
 'Cook' 'Winona' 'Beltrami' 'Pepin' 'Pope' 'Martin' 'Burnett' 'Mower'
 'Murray' 'MilleLacs' 'Rusk' 'Dunn' 'Carlton' 'Clearwater' 'Dodge'
 'Kossuth' 'Freeborn' 'Ringgold' 'Steele' 'Fillmore' 'Lyon'
 'LakeoftheWoods' 'Kanabec' 'Houston' 'Roberts' 'Wadena' 'BigStone'
 'Nobles' 'Poweshiek' 'Waseca' 'Dallas' 'Grant' 'DesMoines' 'Rock'
 'Marion' 'Redwood' 'Cottonwood' 'Mcleod' 'PaloAlto' 'Hardin' 'Carroll'
 'Brown' 'Pottawattamie' 'Warren'

In [ ]:
# Keep twin cities counties
options = ['Hennepin', 'Ramsey', 'Dakota', 'Anoka', 'Washington', 'Scott', 'Wright', 'Carver', 'Sherburne', 'Chisago', 'Isanti', 'LeSueur', 'MilleLacs'] 
df_properties = df_properties[df_properties['county'].isin(options)]

In [ ]:
#check county unique values
print(df_properties.county.unique())
print(df_properties.shape)

['Hennepin' 'Washington' 'Dakota' 'Anoka' 'Scott' 'Ramsey' 'Wright'
 'Chisago' 'Sherburne' 'Carver' 'Isanti' 'LeSueur' 'MilleLacs']
(62993, 39)


## Missing Data


Now that we have gotten rid of redundant data we will now deal with null values. 

For this dataset we will just keep the data that does not have null values.

In [ ]:
df_properties_all_counties=df_properties.copy()
df_properties_all_counties.shape

(62993, 39)

In [ ]:
#replaces na values of existing lisitings with their probable value

df_properties_all_counties['num_bath']=df_properties_all_counties['num_bath'].fillna("0.0")  
df_properties_all_counties['num_bed']=df_properties_all_counties['num_bed'].fillna("0.0")
df_properties_all_counties['out_buildings']=df_properties_all_counties['out_buildings'].fillna("None") 
df_properties_all_counties['association_fee']=df_properties_all_counties['association_fee'].fillna(0)
df_properties_all_counties['appliances']=df_properties_all_counties['appliances'].fillna("None")
df_properties_all_counties['num_fireplaces']=df_properties_all_counties['num_fireplaces'].fillna(0)
df_properties_all_counties['amenities']=df_properties_all_counties['amenities'].fillna("None")
df_properties_all_counties['handicap']=df_properties_all_counties['handicap'].fillna("None")
df_properties_all_counties['pool']=df_properties_all_counties['pool'].fillna("None")
df_properties_all_counties['dining']=df_properties_all_counties['dining'].fillna("None")
df_properties_all_counties['family_room']=df_properties_all_counties['family_room'].fillna("None")

In [ ]:
#now we can use dropna to get rid of the rows that are entirely nas without dropping rows with only some data missing
df_properties_all_counties=df_properties_all_counties.dropna()
df_properties_all_counties.describe

<bound method NDFrame.describe of             pid   sell_date  ...                     sewer                water
1       5708235  03/15/2021  ...       CitySewer/Connected  CityWater/Connected
2       5618755  11/05/2020  ...              PrivateSewer                 Well
3       5501312  04/24/2020  ...  PrivateSewer:HoldingTank                 Well
5       5628081  10/14/2020  ...  PrivateSewer:HoldingTank                 Well
7       5570154  05/22/2020  ...       CitySewer/Connected  CityWater/Connected
...         ...         ...  ...                       ...                  ...
135655  5579898  08/07/2020  ...                   Private                 None
135815  5696193  02/02/2021  ...              PrivateSewer              Private
136118  5679751  01/19/2021  ...       CitySewer/Connected  CityWater/Connected
136692  5553500  06/19/2020  ...       CitySewer/Connected  CityWater/Connected
138003  5671913  12/04/2020  ...       CitySewer/Connected  CityWater/Connected

[5902

### 2.3.3 Splitting long string data columns

In [ ]:
# Change types into something we can use and manipulate

df_properties['pid'] = df_properties['pid'].astype('int')
df_properties['sell_date'] = df_properties['sell_date'].astype('str')

df_properties['city'] = df_properties['city'].astype('str')
df_properties['county'] = df_properties['county'].astype('str')
df_properties['property_type'] = df_properties['property_type'].astype('str')
df_properties['appliances'] = df_properties['appliances'].astype('str')
df_properties['basement_details'] = df_properties['basement_details'].astype('str')
df_properties['common_wall'] = df_properties['common_wall'].astype('str')
df_properties['heating'] = df_properties['heating'].astype('str')
df_properties['ac'] = df_properties['ac'].astype('str')
df_properties['amenities'] = df_properties['amenities'].astype('str')
df_properties['ext_material'] = df_properties['ext_material'].astype('str')
df_properties['handicap'] = df_properties['handicap'].astype('str')
df_properties['out_buildings'] = df_properties['out_buildings'].astype('str')
df_properties['pool'] = df_properties['pool'].astype('str')

df_properties['dining'] = df_properties['dining'].astype('str')
df_properties['family_room'] = df_properties['family_room'].astype('str')
df_properties['zoning'] = df_properties['zoning'].astype('str')

df_properties['fuel'] = df_properties['fuel'].astype('str')
df_properties['sewer'] = df_properties['sewer'].astype('str')
df_properties['water'] = df_properties['water'].astype('str')


In [ ]:
df_properties_all_counties.shape

(59028, 39)

## Clean 'heating' column

There a lot of ways to clean up this column. We have decided to list the main types of heating and create their own columns. This is important because it keeps more of the data than taking the first entry within a row.

In [ ]:
df_properties_all_counties.shape

(59028, 39)

In [ ]:
# We will categorize all unique values into these 10 categories
forced_air = []
boiler = []
hot_water = []
radiant = []
heat_pump = []
dual = []
ductless_mini_split = []
base_board = []
geothermal = []
other = []

In [ ]:
#condense options for heating
for i in df_properties_all_counties.index.values:
  if 'ForcedAir' in df_properties_all_counties['heating'][i]:
    forced_air.append(1)
  else:
    forced_air.append(0)
  if 'Boiler' in df_properties_all_counties['heating'][i]:
    boiler.append(1)
  else:
    boiler.append(0)
  if 'HotWater' in df_properties_all_counties['heating'][i]:
    hot_water.append(1)
  else:
    hot_water.append(0)
  if 'Radiant' in df_properties_all_counties['heating'][i]:
    radiant.append(1)
  else:
    radiant.append(0)
  if 'HeatPump' in df_properties_all_counties['heating'][i]:
    heat_pump.append(1)
  else:
    heat_pump.append(0)
  if 'Dual' in df_properties_all_counties['heating'][i]:
    dual.append(1)
  else:
    dual.append(0)
  if 'DuctlessMini-Split' in df_properties_all_counties['heating'][i]:
    ductless_mini_split.append(1)
  else:
    ductless_mini_split.append(0)
  if 'Baseboard' in df_properties_all_counties['heating'][i]:
    base_board.append(1)
  else:
    base_board.append(0)
  if 'Geothermal' in df_properties_all_counties['heating'][i]:
    geothermal.append(1)
  else:
    geothermal.append(0)
  if 'Other' in df_properties_all_counties['heating'][i]:
    other.append(1)
  else:
    other.append(0)


In [ ]:
# Copy lists to new columns
df_properties_all_counties['forced_air_heater'] = forced_air
df_properties_all_counties['boiler_heater'] = boiler
df_properties_all_counties['hot_water_heater'] = hot_water
df_properties_all_counties['radiant_heat'] = radiant
df_properties_all_counties['heat_pump'] = heat_pump
df_properties_all_counties['dual_heat'] = dual
df_properties_all_counties['ductless_mini_split_heat'] = ductless_mini_split
df_properties_all_counties['baseboard_heater'] = base_board
df_properties_all_counties['geothermal_heat'] = geothermal
df_properties_all_counties['other_heat'] = other

In [ ]:
# delete heating column
del df_properties_all_counties['heating']

In [ ]:
df_properties_all_counties.shape

(59028, 48)

## Clean 'Pool' Column

Because we can't really split up this data into it's own columns we need to ensure the naming description is in a predictable format so there are no duplicates. 

To achieve this we will do the following steps:
1. Go through each entry
2. Split the entry into a list and alphabetize it
3. Make all the entry's lowercase
4. Replace words that mean the same (example: outdoorpool is the same as outdoor)
5. Combine them back into one string

In [ ]:
# copy data frame for pool cleanup
df_properties_pool_clean = df_properties_all_counties.copy()
df_properties_pool_clean = df_properties_pool_clean[['pid', 'pool']]

In [ ]:
pool_list = []

In [ ]:
# Clean pool list. Condenses list down to be able to one hot encode
for i in range(0, len(df_properties_pool_clean)):
  pool_list.append(sorted(df_properties_pool_clean.iloc[i][1].split(":"))) 
  if pool_list[i] is not None:
    pool_list[i] = [x.lower() for x in pool_list[i]]
  if "outdoorpool" in pool_list[i]:
    pool_list[i] = [sub.replace("outdoorpool", "outdoor") for sub in pool_list[i]]
  pool_list[i] = ':'.join(pool_list[i])

In [ ]:
# Ensure lengths are the same
print(len(pool_list))

59028


In [ ]:
df_properties_all_counties['pool'] = pool_list

In [ ]:
df_properties_all_counties.shape

(59028, 48)

##  Clean 'water' Column

In [ ]:
# copy data frame
df_properties_water_clean = df_properties_all_counties.copy()
df_properties_water_clean = df_properties_water_clean[['pid', 'water']]

In [ ]:
# df_properties_water_clean.water.unique()

In [ ]:
water = []

In [ ]:
#condense options for water sources
for i in df_properties_water_clean.index.values:
  if 'City' in df_properties_water_clean['water'][i]:
    water.append('city_water')
  elif 'Private' in df_properties_water_clean['water'][i]:
    water.append('private_water')
  elif 'Well' in df_properties_water_clean['water'][i]:
    water.append('private_water')   
  elif '4' in df_properties_water_clean['water'][i]:
    water.append('private_water') 
  elif 'Drilled' in df_properties_water_clean['water'][i]:
    water.append('private_water') 
  elif 'Shared' in df_properties_water_clean['water'][i]:
    water.append('shared_water')
  else:
    water.append('n/a')

In [ ]:
print(water)
len(water)

['city_water', 'private_water', 'private_water', 'private_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'private_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'private_water', 'city_water', 'city_water', 'city_water', 'private_water', 'private_water', 'city_water', 'city_water', 'private_water', 'shared_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'private_water', 'city_water', 'city_water', 'city_water', 'private_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'city_water', 'private_water', 'city_water', 'city_water', 'city_water', 'private_water', 'city_water', 'city_water', 'private_water', 'city_

59028

In [ ]:
df_properties_all_counties['water'] = water

In [ ]:
df_properties_all_counties.water.unique()

array(['city_water', 'private_water', 'shared_water', 'n/a'], dtype=object)

In [ ]:
df_properties_all_counties.shape

(59028, 48)

## Clean 'sewer' Column

In [ ]:
# copy data frame
df_properties_sewer_clean = df_properties_all_counties.copy()
df_properties_sewer_clean = df_properties_sewer_clean[['pid', 'sewer']]

In [ ]:
sewer = []

In [ ]:
#condense options for sewer
for i in df_properties_sewer_clean.index.values:
  if 'City' in df_properties_sewer_clean['sewer'][i]:
    sewer.append('city_sewer')
  elif 'Private' in df_properties_sewer_clean['sewer'][i]:
    sewer.append('private_sewer')
  elif 'Mound' in df_properties_sewer_clean['sewer'][i]:
    sewer.append('private_sewer')
  elif 'Holding' in df_properties_sewer_clean['sewer'][i]:
    sewer.append('private_sewer')
  elif 'Tank' in df_properties_sewer_clean['sewer'][i]:
    sewer.append('private_sewer')
  elif 'Outhouse' in df_properties_sewer_clean['sewer'][i]:
    sewer.append('private_sewer')
  elif 'Shared' in df_properties_sewer_clean['sewer'][i]:
    sewer.append('shared_sewer')
  elif 'Other' in df_properties_sewer_clean['sewer'][i]:
    sewer.append('other')
  else:
    sewer.append('none')

In [ ]:
print(sewer)
len(sewer)

['city_sewer', 'private_sewer', 'private_sewer', 'private_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'shared_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'city_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'city_sewer', 'city_sewer', 'private_sewer', 'city_sew

59028

In [ ]:
df_properties_all_counties['sewer'] = sewer

In [ ]:
df_properties_all_counties.sewer.unique()

array(['city_sewer', 'private_sewer', 'shared_sewer', 'other', 'none'],
      dtype=object)

In [ ]:
df_properties_all_counties.shape

(59028, 48)

## Clean 'fuel' Column

In [ ]:
# copy data frame
df_properties_fuel_clean = df_properties_all_counties.copy()
df_properties_fuel_clean = df_properties_fuel_clean[['pid', 'fuel']]

In [ ]:
# df_properties_fuel_clean.fuel.unique()

In [ ]:
fuel = []

In [ ]:
#condense options
for i in df_properties_fuel_clean.index.values:
  if 'Natural' in df_properties_fuel_clean['fuel'][i]:
    fuel.append('natural_gas')
  elif 'Propane' in df_properties_fuel_clean['fuel'][i]:
    fuel.append('propane')
  elif 'Electric' in df_properties_fuel_clean['fuel'][i]:
    fuel.append('electric')
  elif 'Solar' in df_properties_fuel_clean['fuel'][i]:
    fuel.append('electric')
  elif 'Oil' in df_properties_fuel_clean['fuel'][i]:
    fuel.append('oil')
  elif 'Wood' in df_properties_fuel_clean['fuel'][i]:
    fuel.append('wood')
  elif 'Pellet' in df_properties_fuel_clean['fuel'][i]:
    fuel.append('wood')
  elif 'Other' in df_properties_fuel_clean['fuel'][i]:
    fuel.append('other')
  else:
    fuel.append('none')

In [ ]:
print(fuel)
len(fuel)

['natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'propane', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'propane', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'natural_gas', 'n

59028

In [ ]:
df_properties_all_counties['fuel'] = fuel

In [ ]:
df_properties_all_counties.fuel.unique()

array(['natural_gas', 'propane', 'electric', 'other', 'oil', 'wood'],
      dtype=object)

In [ ]:
df_properties_all_counties.shape

(59028, 48)

## Clean 'Ext_Material' Column

In [ ]:
# copy data frame
df_properties_ext_material_clean = df_properties_all_counties.copy()
df_properties_ext_material_clean = df_properties_ext_material_clean[['pid', 'ext_material']]

In [ ]:
ext_material = []

In [ ]:
#condense options
for i in df_properties_ext_material_clean.index.values:
  if 'Vinyl' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('vinyl')
  elif 'Stucco' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('stucco')
  elif 'Cement' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('cement')
  elif 'Concrete' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('cement')
  elif 'Brick' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('brick')
  elif 'Wood' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('wood')
  elif 'Shake' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('wood')
  elif 'Fiber' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('wood')
  elif 'Metal' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('metal')
  elif 'Aluminum' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('metal')
  elif 'Other' in df_properties_ext_material_clean['ext_material'][i]:
    ext_material.append('other')  
  else:
    ext_material.append('none')

In [ ]:
print(ext_material)

['stucco', 'brick', 'stucco', 'brick', 'vinyl', 'vinyl', 'vinyl', 'cement', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'brick', 'stucco', 'vinyl', 'cement', 'cement', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'stucco', 'vinyl', 'vinyl', 'vinyl', 'brick', 'vinyl', 'vinyl', 'vinyl', 'brick', 'vinyl', 'vinyl', 'vinyl', 'cement', 'vinyl', 'vinyl', 'none', 'metal', 'wood', 'brick', 'stucco', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'wood', 'vinyl', 'brick', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'wood', 'brick', 'vinyl', 'wood', 'vinyl', 'vinyl', 'vinyl', 'wood', 'vinyl', 'brick', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'wood', 'vinyl', 'stucco', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'stucco', 'brick', 'vinyl', 'vinyl', 'vinyl', 'wood', 'vinyl', 'vinyl', 'vinyl', 'vinyl', 'stucco', 'wood', 'vinyl', 'vinyl', 'vinyl', 'cement', 'vinyl', 'vinyl', 'none', 'vinyl', 'stucco', 'wood', 'vinyl', 'vinyl', 'wood', 'stucco', 'vinyl', 'stucc

In [ ]:
df_properties_all_counties['ext_material'] = ext_material

In [ ]:
df_properties_all_counties.ext_material.unique()

array(['stucco', 'brick', 'vinyl', 'cement', 'none', 'metal', 'wood',
       'other'], dtype=object)

In [ ]:
df_properties_all_counties.shape

(59028, 48)

## Clean 'AC' Column

In [ ]:
# copy data frame
df_properties_ac_clean = df_properties_all_counties.copy()
df_properties_ac_clean = df_properties_ac_clean[['pid', 'ac']]

In [ ]:
ac = []

In [ ]:
#condense options
for i in df_properties_ac_clean.index.values:
  if 'Central' in df_properties_ac_clean['ac'][i]:
    ac.append('central_air')
  elif 'Duct' in df_properties_ac_clean['ac'][i]:
    ac.append('ductless_mini-split')
  elif 'Window' in df_properties_ac_clean['ac'][i]:
    ac.append('window/wall_unit')
  elif 'Wall' in df_properties_ac_clean['ac'][i]:
    ac.append('window/wall_unit')
  elif 'Geothermal' in df_properties_ac_clean['ac'][i]:
    ac.append('geothermal')
  elif 'Whole' in df_properties_ac_clean['ac'][i]:
    ac.append('whole_house_fan')
  elif 'Other' in df_properties_ac_clean['ac'][i]:
    ac.append('other')
  else:
    ac.append('none')

In [ ]:
print(ac)

['central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_air', 'central_

In [ ]:
df_properties_all_counties['ac'] = ac

In [ ]:
df_properties_all_counties.ac.unique()

array(['central_air', 'ductless_mini-split', 'window/wall_unit',
       'geothermal', 'none', 'whole_house_fan'], dtype=object)

In [ ]:
df_properties_all_counties.shape

(59028, 48)

## Clean 'Family_Room' Column

In [ ]:
# copy data frame
df_properties_family_room_clean = df_properties_all_counties.copy()
df_properties_family_room_clean = df_properties_family_room_clean[['pid', 'family_room']]

In [ ]:
lower_level_family_room = []
main_level_family_room = []
loft = []
entertainment_media_center = []
vaulted_ceiling = []
great_room = []

In [ ]:
#condense options for heating
for i in df_properties_family_room_clean.index.values:
  if 'Lower' in df_properties_family_room_clean['family_room'][i]:
    lower_level_family_room.append(1)
  else:
    lower_level_family_room.append(0)
  if 'Main' in df_properties_family_room_clean['family_room'][i]:
    main_level_family_room.append(1)
  else:
    main_level_family_room.append(0)
  if 'Loft' in df_properties_family_room_clean['family_room'][i]:
    loft.append(1)
  else:
    loft.append(0)
  if 'Entertainment' in df_properties_family_room_clean['family_room'][i]:
    entertainment_media_center.append(1)
  else:
    entertainment_media_center.append(0)
  if 'Vault' in df_properties_family_room_clean['family_room'][i]:
    vaulted_ceiling.append(1)
  else:
    vaulted_ceiling.append(0)
  if 'Great' in df_properties_family_room_clean['family_room'][i]:
    great_room.append(1)
  else:
    great_room.append(0)

In [ ]:
df_properties_all_counties['lower_level_family_room'] = lower_level_family_room
df_properties_all_counties['main_level_family_room'] = main_level_family_room
df_properties_all_counties['loft'] = loft
df_properties_all_counties['entertainment/media_center'] = entertainment_media_center
df_properties_all_counties['vaulted_ceilings'] = vaulted_ceiling
df_properties_all_counties['great_room'] = great_room

In [ ]:
del df_properties_all_counties['family_room']

In [ ]:
df_properties_all_counties.shape

(59028, 53)

## Clean 'Dining' Column

In [ ]:
# copy data frame
df_properties_dining_clean = df_properties_all_counties.copy()
df_properties_dining_clean = df_properties_dining_clean[['pid', 'dining']]

In [ ]:
formal_dining_room = []
informal_dining_room = []
breakfast_breakfast_bar = []

In [ ]:
#condense options for heating
for i in df_properties_dining_clean.index.values:
  if "Formal" in df_properties_dining_clean['dining'][i] or "Seperated" in df_properties_dining_clean['dining'][i]:
    formal_dining_room.append(1)
  else:
    formal_dining_room.append(0)
  if 'Breakfast' in df_properties_dining_clean['dining'][i]:
    breakfast_breakfast_bar.append(1)
  else:
    breakfast_breakfast_bar.append(0)
  if 'Informal' in df_properties_dining_clean['dining'][i] or 'Living' in df_properties_dining_clean['dining'][i] or 'Kitchen' in df_properties_dining_clean['dining'][i]:
    informal_dining_room.append(1)
  else:
    informal_dining_room.append(0)

In [ ]:
df_properties_all_counties['formal_dining_room'] = formal_dining_room
df_properties_all_counties['informal_dining_room'] = informal_dining_room
df_properties_all_counties['breakfast_area/breakfast_bar'] = breakfast_breakfast_bar

In [ ]:
del df_properties_all_counties['dining']

In [ ]:
df_properties_all_counties.shape

(59028, 55)

## Clean 'Out_Building' Column

In [ ]:
# copy data frame
df_properties_out_buildings_clean = df_properties_all_counties.copy()
df_properties_out_buildings_clean = df_properties_out_buildings_clean[['pid', 'out_buildings']]

In [ ]:
shed = []
additional_garage = []
barn = []
coop = []
hottub = []
workshop = []
studio = []
gazebo = []
boat_house = []
dog_kennel = []
guest_house = []
sauna = []
stable = []
arena = []
green_house = []
other_out_buildings = []

In [ ]:
#condense options
for i in df_properties_out_buildings_clean.index.values:
  if "Shed" in df_properties_out_buildings_clean['out_buildings'][i]:
    shed.append(1)
  else:
    shed.append(0)
  if "Garage" in df_properties_out_buildings_clean['out_buildings'][i]:
    additional_garage.append(1)
  else:
    additional_garage.append(0)
  if "Barn" in df_properties_out_buildings_clean['out_buildings'][i] or "Pole" in df_properties_out_buildings_clean['out_buildings'][i] or "Lean" in df_properties_out_buildings_clean['out_buildings'][i]:
    barn.append(1)
  else:
    barn.append(0)
  if "Chicken" in df_properties_out_buildings_clean['out_buildings'][i] or "Hen" in df_properties_out_buildings_clean['out_buildings'][i]:
    coop.append(1)
  else:
    coop.append(0)
  if "Hot" in df_properties_out_buildings_clean['out_buildings'][i]:
    hottub.append(1)
  else:
    hottub.append(0)
  if "Shop" in df_properties_out_buildings_clean['out_buildings'][i] or "Tack" in df_properties_out_buildings_clean['out_buildings'][i]:
    workshop.append(1)
  else:
    workshop.append(0)
  if "Studio" in df_properties_out_buildings_clean['out_buildings'][i]:
    studio.append(1)
  else:
    studio.append(0)
  if "Gazebo" in df_properties_out_buildings_clean['out_buildings'][i]:
    gazebo.append(1)
  else:
    gazebo.append(0)
  if "Boat" in df_properties_out_buildings_clean['out_buildings'][i]:
    boat_house.append(1)
  else:
    boat_house.append(0)
  if "Dog" in df_properties_out_buildings_clean['out_buildings'][i]:
    dog_kennel.append(1)
  else:
    dog_kennel.append(0)
  if "Guest" in df_properties_out_buildings_clean['out_buildings'][i] or "Bunk" in df_properties_out_buildings_clean['out_buildings'][i]:
    guest_house.append(1)
  else:
    guest_house.append(0)
  if "Sauna" in df_properties_out_buildings_clean['out_buildings'][i]:
    sauna.append(1)
  else:
    sauna.append(0)
  if "Stable" in df_properties_out_buildings_clean['out_buildings'][i]:
    stable.append(1)
  else:
    stable.append(0)
  if "Arena" in df_properties_out_buildings_clean['out_buildings'][i]:
    arena.append(1)
  else:
    arena.append(0)
  if "Green" in df_properties_out_buildings_clean['out_buildings'][i]:
    green_house.append(1)
  else:
    green_house.append(0)


In [ ]:
df_properties_all_counties['shed'] = shed
df_properties_all_counties['additional_garage'] = additional_garage
df_properties_all_counties['barn'] = barn
df_properties_all_counties['chicken_coop'] = coop
df_properties_all_counties['hottub'] = hottub
df_properties_all_counties['workshop'] = workshop
df_properties_all_counties['studio'] = studio
df_properties_all_counties['gazebo'] = gazebo
df_properties_all_counties['boat_house'] = boat_house
df_properties_all_counties['dog_kennel'] = dog_kennel
df_properties_all_counties['guest_house'] = guest_house
df_properties_all_counties['sauna'] = sauna
df_properties_all_counties['stable'] = stable
df_properties_all_counties['arena'] = arena
df_properties_all_counties['green_house'] = green_house

In [ ]:
del df_properties_all_counties['out_buildings']

In [ ]:
df_properties_all_counties.shape

(59028, 69)

## Clean 'Handicap' Column

In [ ]:
# copy data frame
df_properties_handicap_clean = df_properties_all_counties.copy()
df_properties_handicap_clean = df_properties_handicap_clean[['pid', 'handicap']]

In [ ]:
door_lever_handles = []
grab_bars_bathroom = []
wheel_chair_partial_full_accessible = []
doors_36in = []
hallways_42in = []
roll_in_showers = []
no_stairs_external = []
no_stairs_internal = []
accessable_elevator = []
stair_lift = []
lowered_cabinents = []
raised_outlets = []
lowered_switches = []
sight_impaired = []

In [ ]:
#condense options
for i in df_properties_handicap_clean.index.values:
  if "Lever" in df_properties_handicap_clean['handicap'][i]:
    door_lever_handles.append(1)
  else:
    door_lever_handles.append(0)
  if "Grab" in df_properties_handicap_clean['handicap'][i]:
    grab_bars_bathroom.append(1)
  else:
    grab_bars_bathroom.append(0)
  if "Wheel" in df_properties_handicap_clean['handicap'][i]:
    wheel_chair_partial_full_accessible.append(1)
  else:
    wheel_chair_partial_full_accessible.append(0)
  if "36" in df_properties_handicap_clean['handicap'][i]:
    doors_36in.append(1)
  else:
    doors_36in.append(0)
  if "42" in df_properties_handicap_clean['handicap'][i]:
    hallways_42in.append(1)
  else:
    hallways_42in.append(0)
  if "Roll" in df_properties_handicap_clean['handicap'][i]:
    roll_in_showers.append(1)
  else:
    roll_in_showers.append(0)
  if "StairsInternal" in df_properties_handicap_clean['handicap'][i]:
    no_stairs_internal.append(1)
  else:
    no_stairs_internal.append(0)
  if "StairsExternal" in df_properties_handicap_clean['handicap'][i]:
    no_stairs_external.append(1)
  else:
    no_stairs_external.append(0)
  if "Elevator" in df_properties_handicap_clean['handicap'][i]:
    accessable_elevator.append(1)
  else:
    accessable_elevator.append(0)
  if "Lift" in df_properties_handicap_clean['handicap'][i]:
    stair_lift.append(1)
  else:
    stair_lift.append(0)
  if "Cabinent" in df_properties_handicap_clean['handicap'][i]:
    lowered_cabinents.append(1)
  else:
    lowered_cabinents.append(0)
  if "Outlets" in df_properties_handicap_clean['handicap'][i]:
    raised_outlets.append(1)
  else:
    raised_outlets.append(0)
  if "Switch" in df_properties_handicap_clean['handicap'][i]:
    lowered_switches.append(1)
  else:
    lowered_switches.append(0)
  if "Sight" in df_properties_handicap_clean['handicap'][i]:
    sight_impaired.append(1)
  else:
    sight_impaired.append(0)

In [ ]:
df_properties_all_counties['lever_door_handles'] = door_lever_handles
df_properties_all_counties['grab_bar_bathroom'] = grab_bars_bathroom
df_properties_all_counties['partially/fully_wheel_chair_accessible'] = wheel_chair_partial_full_accessible
df_properties_all_counties['36in_doors'] = doors_36in
df_properties_all_counties['42in_hallways'] = hallways_42in
df_properties_all_counties['roll_in_shower'] = roll_in_showers
df_properties_all_counties['no_external_stairs'] = no_stairs_external
df_properties_all_counties['no_internal_stairs'] = no_stairs_internal
df_properties_all_counties['accessibility_elevator'] = accessable_elevator
df_properties_all_counties['stair_lift'] = stair_lift
df_properties_all_counties['lowered_cabinents'] = lowered_cabinents
df_properties_all_counties['raised_outlets'] = raised_outlets
df_properties_all_counties['lowered_switches'] = lowered_switches
df_properties_all_counties['sight_impaired'] = sight_impaired

In [ ]:
del df_properties_all_counties['handicap']

In [ ]:
df_properties_all_counties.shape

(59028, 82)

## Clean 'Basement_Details' Column

In [ ]:
# copy data frame for description analysis
df_properties_basement_details_clean = df_properties_all_counties.copy()
df_properties_basement_details_clean = df_properties_basement_details_clean[['pid', 'basement_details']]

In [ ]:
finished = []
paritally_finished = []
unfinished = []
walk_out = []
sumppump = []
drain_tiled = []
daylight_look_out_windows = []
egress_windows = []
full_basement = []

In [ ]:
for i in df_properties_basement_details_clean.index.values:
  if "Finished" in df_properties_basement_details_clean['basement_details'][i]:
    if "Partial" in df_properties_basement_details_clean["basement_details"][i]:
      paritally_finished.append(1)
      unfinished.append(0)
      finished.append(0)
    elif "Unfinished" in df_properties_basement_details_clean['basement_details'][i]:
      unfinished.append(1)
      paritally_finished.append(0)
      finished.append(0)
    else:
      finished.append(1)
      paritally_finished.append(0)
      unfinished.append(0)
  else:
    finished.append(0)
    unfinished.append(0)
    paritally_finished.append(0)
  if "Walk" in df_properties_basement_details_clean['basement_details'][i]:
    walk_out.append(1)
  else:
    walk_out.append(0)
  if "Sump" in df_properties_basement_details_clean['basement_details'][i]:
    sumppump.append(1)
  else:
    sumppump.append(0)
  if "Drain" in df_properties_basement_details_clean['basement_details'][i]:
    drain_tiled.append(1)
  else:
    drain_tiled.append(0)
  if "Daylight" in df_properties_basement_details_clean['basement_details'][i]:
    daylight_look_out_windows.append(1)
  else:
    daylight_look_out_windows.append(0)
  if "Egress" in df_properties_basement_details_clean['basement_details'][i]:
    egress_windows.append(1)
  else:
    egress_windows.append(0)
  if "Full" in df_properties_basement_details_clean['basement_details'][i]:
    full_basement.append(1)
  else:
    full_basement.append(0)

In [ ]:
df_properties_all_counties['finished_basement'] = finished
df_properties_all_counties['partially_finished_basement'] = paritally_finished
df_properties_all_counties['unfinished_basement'] = unfinished
df_properties_all_counties['walk_out_basement'] = walk_out
df_properties_all_counties['sump_pump_basement'] = sumppump
df_properties_all_counties['drain_tiled_basement'] = drain_tiled
df_properties_all_counties['daylight_look_out_windows_basement'] = daylight_look_out_windows
df_properties_all_counties['egress_windows_basement'] = egress_windows
df_properties_all_counties['full_basement'] = full_basement

In [ ]:
del df_properties_all_counties['basement_details']

In [ ]:
df_properties_all_counties.shape

(59028, 90)

## Clean 'Amenities' Column

In [ ]:
# copy data frame
df_properties_amenities_clean = df_properties_all_counties.copy()
df_properties_amenities_clean = df_properties_amenities_clean[['pid', 'amenities']]

In [ ]:
df_properties_amenities_clean.amenities.unique()

array(['Deck:Patio:Porch:NaturalWoodwork:KitchenWindow:KitchenCenterIsland:VaultedCeiling(s):HardwoodFloors:TiledFloors:PaneledDoors:FrenchDoors:Walk-InCloset:MasterBedroomWalk-InCloset:MainFloorMasterBedroom:WetBar:ExerciseRoom:SunRoom:SecuritySystem:In-GroundSprinkler:PanoramicView',
       'NaturalWoodwork:KitchenWindow:HardwoodFloors:TiledFloors:Washer/DryerHookup',
       'Deck:Patio:NaturalWoodwork:HotTub:Sauna:KitchenCenterIsland:CeilingFan(s):Walk-InCloset:MasterBedroomWalk-InCloset:MainFloorMasterBedroom:ExerciseRoom:SunRoom:Washer/DryerHookup:PanoramicView:OutdoorKitchen',
       ..., 'CeilingFan(s):TiledFloors:MasterBedroomWalk-InCloset:Cable',
       'IndoorSprinklers:NaturalWoodwork:Walk-InCloset',
       'Deck:KitchenWindow:LocalAreaNetwork:MultiplePhoneLines'],
      dtype=object)

In [ ]:
deck = []
exercise_room = []
french_doors = []
hardwood_floors = []
sprinkler = []
kitchen_island = []
porch = []
security_system = []
sunroom = []
washer_dryer_hookup = []
balcony = []
walk_in_closet = []
wet_bar = []
cable = []
patio = []
ceiling_fans = []
natural_wood_work = []
dock = []
tiled_floors = []
city_view = []
boat_slip = []
sky_light = []
attic = []

In [ ]:
for i in df_properties_amenities_clean.index.values:
  if "Deck" in df_properties_amenities_clean['amenities'][i]:
    deck.append(1)
  else:
    deck.append(0)
  if "Exercise" in df_properties_amenities_clean['amenities'][i]:
    exercise_room.append(1)
  else:
    exercise_room.append(0)
  if "French" in df_properties_amenities_clean['amenities'][i]:
    french_doors.append(1)
  else:
    french_doors.append(0)
  if "Hardwood" in df_properties_amenities_clean['amenities'][i]:
    hardwood_floors.append(1)
  else:
    hardwood_floors.append(0)
  if "Sprinkler" in df_properties_amenities_clean['amenities'][i]:
    sprinkler.append(1)
  else:
    sprinkler.append(0)
  if "Island" in df_properties_amenities_clean['amenities'][i]:
    kitchen_island.append(1)
  else:
    kitchen_island.append(0)
  if "Porch" in df_properties_amenities_clean['amenities'][i]:
    porch.append(1)
  else:
    porch.append(0)
  if "Security" in df_properties_amenities_clean['amenities'][i]:
    security_system.append(1)
  else:
    security_system.append(0)
  if "Sun" in df_properties_amenities_clean['amenities'][i]:
    sunroom.append(1)
  else:
    sunroom.append(0)
  if "Washer" in df_properties_amenities_clean['amenities'][i]:
    washer_dryer_hookup.append(1)
  else:
    washer_dryer_hookup.append(0)
  if "Balcony" in df_properties_amenities_clean['amenities'][i]:
    balcony.append(1)
  else:
    balcony.append(0)
  if "Walk-InCloset" in df_properties_amenities_clean['amenities'][i]:
    walk_in_closet.append(1)
  else:
    walk_in_closet.append(0)
  if "WetBar" in df_properties_amenities_clean['amenities'][i]:
    wet_bar.append(1)
  else:
    wet_bar.append(0)
  if "Cable" in df_properties_amenities_clean['amenities'][i]:
    cable.append(1)
  else:
    cable.append(0)
  if "Patio" in df_properties_amenities_clean['amenities'][i]:
    patio.append(1)
  else:
    patio.append(0)
  if "CeilingFan" in df_properties_amenities_clean['amenities'][i]:
    ceiling_fans.append(1)
  else:
    ceiling_fans.append(0)
  if "Natural" in df_properties_amenities_clean['amenities'][i]:
    natural_wood_work.append(1)
  else:
    natural_wood_work.append(0)
  if "Dock" in df_properties_amenities_clean['amenities'][i]:
    dock.append(1)
  else:
    dock.append(0)
  if "Tiled" in df_properties_amenities_clean['amenities'][i]:
    tiled_floors.append(1)
  else:
    tiled_floors.append(0)
  if "CityView" in df_properties_amenities_clean['amenities'][i]:
    city_view.append(1)
  else:
    city_view.append(0)
  if "BoatSlip" in df_properties_amenities_clean['amenities'][i]:
    boat_slip.append(1)
  else:
    boat_slip.append(0)
  if "SkyLight" in df_properties_amenities_clean['amenities'][i]:
    sky_light.append(1)
  else:
    sky_light.append(0)
  if "Attic" in df_properties_amenities_clean['amenities'][i]:
    attic.append(1)
  else:
    attic.append(0)


In [ ]:
df_properties_all_counties['deck'] = deck
df_properties_all_counties['exercise_room'] = exercise_room
df_properties_all_counties['french_doors'] = french_doors
df_properties_all_counties['hardwood_floors'] = hardwood_floors
df_properties_all_counties['sprinkler_system'] = sprinkler
df_properties_all_counties['kitchen_island'] = kitchen_island
df_properties_all_counties['porch'] = porch
df_properties_all_counties['security_system'] = security_system
df_properties_all_counties['sunroom'] = sunroom
df_properties_all_counties['washer_dryer_hookup'] = washer_dryer_hookup
df_properties_all_counties['balcony'] = balcony
df_properties_all_counties['walk_in_closets'] = walk_in_closet
df_properties_all_counties['wet_bar'] = wet_bar
df_properties_all_counties['cable_hookup'] = cable
df_properties_all_counties['patio'] = patio
df_properties_all_counties['ceiling_fans'] = ceiling_fans
df_properties_all_counties['natural_wood_work'] = natural_wood_work
df_properties_all_counties['dock'] = dock
df_properties_all_counties['tiled_floor'] = tiled_floors
df_properties_all_counties['city_view'] = city_view
df_properties_all_counties['boat_slip'] = boat_slip
df_properties_all_counties['sky_light'] = sky_light
df_properties_all_counties['attic'] = attic


In [ ]:
del df_properties_all_counties['amenities']

In [ ]:
df_properties_all_counties.shape

(59028, 112)

## Clean 'Appliances' Column

In [ ]:
appliance_df=df_properties_all_counties.copy()
appliance_df = appliance_df[['pid', 'appliances']]

In [ ]:
appliance_df.sort_values(by="appliances", key=lambda x: x.str.len())

,pid,appliances
138003,5671913,None
88789,5684462,None
88808,5666922,None
88811,5709282,None
88819,5689603,None
...,...,...
1187,5612526,Cooktop:WallOven:Microwave:ExhaustFan/Hood:Dishwasher:TrashCompactor:Refrigerator:Washer:Dryer:WaterSoftener-Owned:GasWaterHeater:WaterOsmosisSystem:Disposal:CentralVacuum:ElectronicAirFilter:Air-To-AirExchanger:FurnaceHumidifier
1957,5651862,Cooktop:WallOven:Microwave:ExhaustFan:Dishwasher:Refrigerator:Freezer:Washer:Dryer:WaterSoftenerOwned:GasWaterHeater:WaterFiltrationSystem:WaterOsmosisSystem:Disposal:CentralVacuum:Air-To-AirExchanger:ElectronicAirFilter:Humidifier
8420,5676618,Range:WallOven:Microwave:ExhaustFan:Dishwasher:TrashCompactor:Refrigerator:Washer:Dryer:WaterSoftenerOwned:ElectricWaterHeater:GasWaterHeater:WaterFiltrationSystem:WaterOsmosisSystem:Disposal:CentralVacuum:Air-To-AirExchanger:Humidifier
12907,5639731,Cooktop:WallOven:Microwave:ExhaustFan:Dishwasher:TrashCompactor:Refrigerator:Freezer:Washer:Dryer:WaterSoftenerOwned:GasWaterHeater:WaterFiltrationSystem:WaterOsmosisSystem:CentralVacuum:ElectronicAirFilter:Air-To-AirExchanger:Humidifier


In [ ]:
appliance_df['appliances']=appliance_df['appliances'].replace({"-Owned" : "Owned"}, regex=True)
appliance_df['appliances']=appliance_df['appliances'].replace({"-Rented" : "Rented"}, regex=True)
appliance_df['appliances']=appliance_df['appliances'].replace({":" : ", "}, regex=True)

In [ ]:
a = list(set(', '.join(appliance_df['appliances']).split(', ')))
print (a)
len(a)

['IndoorGrill', 'Dryer', 'CentralVacuum', 'Humidifier', 'ExhaustFan/Hood', 'FurnaceHumidifier', 'Microwave', 'TanklessWaterHeater', 'TrashCompactor', 'WallOven', 'Air-To-AirExchanger', 'None', 'WoodWaterHeater', 'ElectronicAirFilter', 'WaterSoftenerOwned', 'Refrigerator', 'Range', 'Washer', 'Freezer', 'OilWaterHeater', 'FuelTankRented', 'FuelTankOwned', 'Disposal', 'WaterSoftenerRented', 'WaterFiltrationSystem', 'Cooktop', 'IronFilter', 'Dishwasher', 'ExhaustFan', 'ElectricWaterHeater', 'GasWaterHeater', 'Other', 'WaterOsmosisSystem']


33

In [ ]:
microwave = []
water_heater = []
disposal = []
exhaust_fan = []
dishwasher = []
other_appliances = []
wall_oven = []
air_to_air_exchange = []
fuel_tank_owned = []
trash_compactor = []
air_filter = []
water_softener_owned = []
refrigerator = []
dryer = []
central_vacuum = []
iron_filter = []
freezer = []
no_appliances = []
water_filter = []
cooktop = []
humidifier = []
range = []
fuel_tank_rented = []
water_softener_rented = []
indoor_grill = []

In [ ]:
for i in appliance_df.index.values:
  if "Microwave" in appliance_df['appliances'][i]:
    microwave.append(1)
  else:
    microwave.append(0)
  if "WoodWaterHeater" in appliance_df['appliances'][i] or "OilWaterHeater" in appliance_df['appliances'][i] or "ElectricWaterHeater" in appliance_df['appliances'][i] or "GasWaterHeater" in appliance_df['appliances'][i] or "TanklessWaterHeater" in appliance_df['appliances'][i]:
    water_heater.append(1)
  else:
    water_heater.append(0)
  if "Disposal" in appliance_df['appliances'][i]:
    disposal.append(1)
  else:
    disposal.append(0)
  if "ExhaustFan/Hood" in appliance_df['appliances'][i] or "ExhaustFan" in appliance_df['appliances'][i]:
    exhaust_fan.append(1)
  else:
    exhaust_fan.append(0)
  if "Dishwasher" in appliance_df['appliances'][i] or "Washer" in appliance_df['appliances'][i]:
    dishwasher.append(1)
  else:
    dishwasher.append(0)
  if "Other" in appliance_df['appliances'][i]:
    other_appliances.append(1)
  else:
    other_appliances.append(0)
  if "WallOven" in appliance_df['appliances'][i]:
    wall_oven.append(1)
  else:
    wall_oven.append(0)
  if "Air-To-AirExchanger" in appliance_df['appliances'][i]:
    air_to_air_exchange.append(1)
  else:
    air_to_air_exchange.append(0)
  if "FuelTankOwned" in appliance_df['appliances'][i]:
    fuel_tank_owned.append(1)
  else:
    fuel_tank_owned.append(0)
  if "TrashCompactor" in appliance_df['appliances'][i]:
    trash_compactor.append(1)
  else:
    trash_compactor.append(0)
  if "ElectronicAirFilter" in appliance_df['appliances'][i]:
    air_filter.append(1)
  else:
    air_filter.append(0)
  if "WaterSoftenerOwned" in appliance_df['appliances'][i]:
    water_softener_owned.append(1)
  else:
    water_softener_owned.append(0)
  if "Refrigerator" in appliance_df['appliances'][i]:
    refrigerator.append(1)
  else:
    refrigerator.append(0)
  if "Dryer" in appliance_df['appliances'][i]:
    dryer.append(1)
  else:
    dryer.append(0)
  if "CentralVacuum" in appliance_df['appliances'][i]:
    central_vacuum.append(1)
  else:
    central_vacuum.append(0)
  if "IronFilter" in appliance_df['appliances'][i]:
    iron_filter.append(1)
  else:
    iron_filter.append(0)
  if "Freezer" in appliance_df['appliances'][i]:
    freezer.append(1)
  else:
    freezer.append(0)
  if "None" in appliance_df['appliances'][i]:
    no_appliances.append(1)
  else:
    no_appliances.append(0)
  if "WaterOsmosisSystem" in appliance_df['appliances'][i] or "WaterFiltrationSystem" in appliance_df['appliances'][i]:
    water_filter.append(1)
  else:
    water_filter.append(0)
  if "Cooktop" in appliance_df['appliances'][i]:
    cooktop.append(1)
  else:
    cooktop.append(0)
  if "FurnaceHumidifier" in appliance_df['appliances'][i] or "Humidifier" in appliance_df['appliances'][i]:
    humidifier.append(1)
  else:
    humidifier.append(0)
  if "Range" in appliance_df['appliances'][i]:
    range.append(1)
  else:
    range.append(0)
  if "FuelTankRented" in appliance_df['appliances'][i]:
    fuel_tank_rented.append(1)
  else:
    fuel_tank_rented.append(0)
  if "WaterSoftenerRented" in appliance_df['appliances'][i]:
    water_softener_rented.append(1)
  else:
    water_softener_rented.append(0)
  if "IndoorGrill" in appliance_df['appliances'][i]:
    indoor_grill.append(1)
  else:
    indoor_grill.append(0)

In [ ]:
df_properties_all_counties['microwave'] = microwave
df_properties_all_counties['water_heater'] = water_heater
df_properties_all_counties['disposal'] = disposal
df_properties_all_counties['exhaust_fan'] = exhaust_fan
df_properties_all_counties['dishwasher'] = dishwasher
df_properties_all_counties['other_appliances'] = other_appliances
df_properties_all_counties['wall_oven'] = wall_oven
df_properties_all_counties['air_to_air_exchange'] = air_to_air_exchange
df_properties_all_counties['fuel_tank_owned'] = fuel_tank_owned
df_properties_all_counties['trash_compactor'] = trash_compactor
df_properties_all_counties['air_filter'] = air_filter
df_properties_all_counties['water_softener_owned'] = water_softener_owned
df_properties_all_counties['refrigerator'] = refrigerator
df_properties_all_counties['dryer'] = dryer
df_properties_all_counties['central_vacuum'] = central_vacuum
df_properties_all_counties['iron_filter'] = iron_filter
df_properties_all_counties['freezer'] = freezer
df_properties_all_counties['no_appliances'] = no_appliances
df_properties_all_counties['water_filter'] = water_filter
df_properties_all_counties['cooktop'] = cooktop
df_properties_all_counties['humidifier'] = humidifier
df_properties_all_counties['range'] = range
df_properties_all_counties['fuel_tank_rented'] = fuel_tank_rented
df_properties_all_counties['water_softener_rented'] = water_softener_rented
df_properties_all_counties['indoor_grill'] = indoor_grill

del df_properties_all_counties['appliances']

In [ ]:
df_properties_all_counties.shape

(59028, 136)

## Inconsistent Data

In order for the data to be consistent, the following will be modified so to match the rest of the data

  - Capitalization
  - Formats standardization (types)
  - Addresses

#### Capitalizations -> lowercase

In [ ]:
# To lowercase
df_properties_all_counties['city'] = df_properties_all_counties['city'].str.lower()
df_properties_all_counties['county'] = df_properties_all_counties['county'].str.lower()
df_properties_all_counties['property_type'] = df_properties_all_counties['property_type'].str.lower()
df_properties_all_counties['common_wall'] = df_properties_all_counties['common_wall'].str.lower()
df_properties_all_counties['ac'] = df_properties_all_counties['ac'].str.lower()
df_properties_all_counties['ext_material'] = df_properties_all_counties['ext_material'].str.lower()
df_properties_all_counties['pool'] = df_properties_all_counties['pool'].str.lower()
df_properties_all_counties['zoning'] = df_properties_all_counties['zoning'].str.lower()
df_properties_all_counties['fuel'] = df_properties_all_counties['fuel'].str.lower()
df_properties_all_counties['sewer'] = df_properties_all_counties['sewer'].str.lower()
df_properties_all_counties['water'] = df_properties_all_counties['water'].str.lower()

## One Hot Encoding:


In [ ]:
# copy data frame
df_properties_one_hot = df_properties_all_counties.copy()

In [ ]:
import patsy

In [ ]:
# Add columns that need to be one hot encoded
columns = ['ac', 'ext_material', 'pool', 'fuel', 'sewer', 'water']

In [ ]:
# One hot encode columns list
for column in columns:
  one_hot_matrix = patsy.dmatrix(column,data=df_properties_one_hot,return_type='dataframe')
  df_properties_one_hot.drop([column], axis=1, inplace=True)
  df_properties_one_hot = df_properties_one_hot.join(one_hot_matrix)
  del df_properties_one_hot['Intercept']
  

In [ ]:
df_properties_one_hot.shape

(59028, 203)

In [ ]:
df_properties_one_hot.head(5)

,pid,sell_date,sell_price,list_price,address_long,zip_code,built,city,county,property_type,num_bath,num_bed,fin_sqft,above_gd_sqft,below_gd_sqft,num_fireplaces,common_wall,num_garage_spaces,foundation_size,lot,zoning,school_district,association_fee,tax_year,annual_taxes,forced_air_heater,boiler_heater,hot_water_heater,radiant_heat,heat_pump,dual_heat,ductless_mini_split_heat,baseboard_heater,geothermal_heat,other_heat,lower_level_family_room,main_level_family_room,loft,entertainment/media_center,vaulted_ceilings,...,pool[T.belowground:heated:indoor:outdoor:shared],pool[T.belowground:heated:indoor:shared],pool[T.belowground:heated:outdoor],pool[T.belowground:heated:outdoor:shared],pool[T.belowground:heated:shared],pool[T.belowground:indoor],pool[T.belowground:indoor:outdoor],pool[T.belowground:indoor:outdoor:shared],pool[T.belowground:indoor:shared],pool[T.belowground:outdoor],pool[T.belowground:outdoor:shared],pool[T.belowground:shared],pool[T.heated],pool[T.heated:indoor],pool[T.heated:indoor:outdoor],pool[T.heated:indoor:outdoor:shared],pool[T.heated:indoor:shared],pool[T.heated:outdoor],pool[T.heated:outdoor:shared],pool[T.heated:shared],pool[T.indoor],pool[T.indoor:outdoor],pool[T.indoor:outdoor:shared],pool[T.indoor:shared],pool[T.none],pool[T.outdoor],pool[T.outdoor:shared],pool[T.shared],fuel[T.natural_gas],fuel[T.oil],fuel[T.other],fuel[T.propane],fuel[T.wood],sewer[T.none],sewer[T.other],sewer[T.private_sewer],sewer[T.shared_sewer],water[T.n/a],water[T.private_water],water[T.shared_water]
1,5708235,03/15/2021,1275000.0,1300000.0,5027-Nob-Hill-Drive-Edina-MN-55439,55439.0,1993.0,edina,hennepin,singlefamily,5,5,5979.0,4234.0,1745.0,3.0,no,3.0,2915.0,0.81,residential-singlefamily,273,0.0,2020.0,19173.0,1,0,0,1,0,0,0,1,0,0,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5618755,11/05/2020,845000.0,849900.0,683-Oakgreen-Avenue-Court-N-West-Lakeland-Twp-MN-55082,55082.0,2001.0,westlakelandtwp,washington,singlefamily,4,4,4437.0,3329.0,1108.0,3.0,no,3.0,1225.0,2.82,residential-singlefamily,834,200.0,2019.0,5169.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,5501312,04/24/2020,825000.0,825000.0,12299-Lucerne-Trail-Lakeville-MN-55044,55044.0,1997.0,lakeville,dakota,singlefamily,4,5,6192.0,2708.0,3484.0,4.0,no,6.0,2708.0,11.16,residential-singlefamily,194,0.0,2020.0,9272.0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5,5628081,10/14/2020,823000.0,850000.0,2875-Rodeo-Drive-NE-Blaine-MN-55449,55449.0,1992.0,blaine,anoka,singlefamily,6,5,8860.0,6910.0,1950.0,5.0,no,4.0,5020.0,11.11,residential-singlefamily,16,0.0,2019.0,11055.0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7,5570154,05/22/2020,737467.0,737467.0,7525-159th-Street-Savage-MN-55378,55378.0,2020.0,savage,scott,singlefamily,5,5,4573.0,3243.0,1330.0,1.0,no,3.0,1632.0,0.00,residential-singlefamily,719,0.0,2020.0,1399.0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
# Save to google drive
df_properties_one_hot.to_csv('cleaned_housing_data.csv')
!cp cleaned_housing_data.csv "drive/My Drive/"